In [20]:
import os
import sys

sys.path.append("../src")

In [21]:
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.utils.data
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
from copy import deepcopy
import configparser

import models
import my_eval
import severson_data
from loss_functions import nll_loss
import random

import joblib

In [22]:
def get_rmse(
    model,
    idxs,
    x,
    y,
    augmented_data,
    seq_length,
    device,
    scaler,
    max_steps=5000,
    use_cycle_counter=True,
):
    """LSTMモデルから予測したサイクル寿命についてRMSEを計算する関数
    """

    if use_cycle_counter:

        supp_val_data = np.hstack(
            [augmented_data[idxs], np.ones((len(idxs), 1)) * np.log(seq_length)]
        )
    else:
        supp_val_data = augmented_data[idxs]

    # seq_length=100とすると最初の100サイクル目までのデータを初期データとする。
    test_seq = x[idxs][:, :seq_length, None].copy()

    with torch.no_grad():
        # 全てのidxsについて、放電容量の予測値がほぼ0になるまで繰り返す。
        while (np.all(test_seq[:, -1] < 1e-3) == False) * (
            test_seq.shape[1] < max_steps
        ):
            # for i in range(max_steps - seq_length):
            supp_val_data_torch = torch.from_numpy(supp_val_data).to(device).float()
            test_seq_torch = (
                torch.from_numpy(test_seq[:, -seq_length:]).to(device).float()
            )
            model.reset_hidden_state()
            (pred_state, _) = model(test_seq_torch, supp_val_data_torch)

            pred_state = torch.from_numpy(
                scaler.inverse_transform(pred_state.cpu().numpy())
            ).to(device)

            # ((x[i, j + seq_len - 5: j + seq_len ]).mean() + 10e-17)
            pred_state = pred_state[:, 0] * (test_seq_torch[:, -1, 0] + 10e-17)
            # 配列の末尾に予測値をつなげる。
            test_seq = np.hstack([test_seq, pred_state.cpu().numpy()[:, None, None]])
            if use_cycle_counter:
                supp_val_data[:, -1] = np.log(np.exp(supp_val_data[:, -1]) + 1)
        # 0になったインデックスをサイクル寿命の予測値とする。
        y_lifetime_pred = (test_seq[:, :, 0] < cutoff_val).argmax(axis=1)

    return np.sqrt((np.square(y[idxs] - y_lifetime_pred)).mean())

In [23]:
train_data = np.load("train.npz")
valid_data = np.load("valid.npz")
data = np.load("data.npz")

train_x, train_y, train_s = train_data["x"], train_data["y"], train_data["s"]
val_x, val_y, val_s = valid_data["x"], valid_data["y"], valid_data["s"]

In [24]:
old_x = data["old_x"]
smoothed_x = data["smoothed_x"]
augmented_data = data["augmented_data"]
y = data["y"]
train_idx = data["train_idx"]
val_idx = data["val_idx"]
test_idx = data["test_idx"]

In [25]:
config = configparser.ConfigParser()
config.read("../sample_config.ini")

save_path = config["PATHS"]["model_path_severson"]
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [26]:
batch_size = 128
num_epochs = 10
sequence_length = 100
dropout = 0
hidden_size_lstm = -1
hidden_size = 32
use_augment = 1
no_covariates = False

seed = 123
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [27]:
# 前処理
min_val = 0.85
max_val = 1.0
capacity_output_scaler = MinMaxScaler(
    (-1, 1),
    clip=False).fit(np.maximum(np.minimum(train_y[:, 0:1], max_val), min_val))

train_y[:, 0:1] = capacity_output_scaler.transform(train_y[:, 0:1])
val_y[:, 0:1] = capacity_output_scaler.transform(val_y[:, 0:1])

torch.manual_seed(seed)
train_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [train_x, train_y, train_s]])
train_loader = DataLoader(train_dataset,
                          batch_size=batch_size,
                          shuffle=True)

val_dataset = TensorDataset(
    *[torch.Tensor(input) for input in [val_x, val_y, val_s]])
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [28]:
# scalerの保存
with open('scaler.pkl', 'wb') as f:
    pkl.dump(capacity_output_scaler, f)

In [32]:
batch_size = 128
num_epochs = 300
sequence_length = 100
dropout = 0
hidden_size_lstm = -1
hidden_size = 32
use_augment = 1
no_covariates = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
for im in range(5):  # アンサンブルのため5個のモデルを作る
    seed = int(42*im)
    
    experiment_name = f"model{im}"
    
    torch.manual_seed(seed)

    #%%

    input_dim = train_x.shape[
        2]  # Number of input features (e.g. discharge capacity)
    num_augment = train_s.shape[1]

    if no_covariates:
        model = models.Uncertain_LSTM_NoCovariate(
            num_in=input_dim,
            num_augment=num_augment,
            num_hidden=hidden_size,
            num_hidden_lstm=hidden_size_lstm,
            seq_len=sequence_length,
            n_layers=2,
        ).to(device)
    else:
        model = models.Uncertain_LSTM(
            num_in=input_dim,
            num_augment=num_augment,
            num_hidden=hidden_size,
            num_hidden_lstm=hidden_size_lstm,
            seq_len=sequence_length,
            n_layers=2,
        ).to(device)

    optimizer = optim.Adam(model.parameters(), )

    training_loss = []
    validation_loss = []

    best_val_loss = 500000

    cur_patience = 0
    max_patience = 5
    patience_delta = 0.0
    best_weights = None
    
    # 学習
    for epoch in range(num_epochs):

        model.train()
        tr_loss = 0

        for batch_idx, (
                input_data,
                y_hat,
                supp_data,
        ) in enumerate(train_loader):
            model.reset_hidden_state()
            input_data = input_data.to(device)
            supp_data = supp_data.to(device)
            y_hat = y_hat.to(device)
            optimizer.zero_grad()
            (state_mean, state_var) = model(input_data, supp_data)

            # loss
            loss_state = nll_loss(y_hat[:, 0], state_mean[:, 0], state_var[:, 0])
            loss = loss_state
            (loss).backward()
            tr_loss += loss.item()
            optimizer.step()

        tr_loss /= len(train_loader.dataset)
        training_loss.append(tr_loss)

        model.eval()
        val_loss = 0
        val_loss_state = 0
        val_loss_lifetime = 0

        with torch.no_grad():
            for batch_idx, (
                    input_data,
                    y_hat,
                    supp_data,
            ) in enumerate(val_loader):
                model.reset_hidden_state()
                input_data = input_data.to(device)
                supp_data = supp_data.to(device)
                y_hat = y_hat.to(device)

                (state_mean, state_var) = model(input_data, supp_data)

                loss_state = nll_loss(y_hat[:, 0], state_mean[:, 0], state_var[:,
                                                                               0])
                loss = loss_state

                val_loss += loss.item()
                val_loss_state += loss_state.item()

        val_loss /= len(val_loader.dataset)
        val_loss_state /= len(val_loader.dataset)

        val_loss_lifetime /= len(val_loader.dataset)
        validation_loss.append(val_loss)

        print("Epoch: %d, Training loss: %1.5f, Validation loss: %1.5f, " % (
            epoch + 1,
            tr_loss,
            val_loss,
        ))

        if val_loss + patience_delta < best_val_loss:
            best_weights = deepcopy(model.state_dict())
            cur_patience = 0
            best_val_loss = val_loss
        else:
            cur_patience += 1
        if cur_patience > max_patience:
            break


    # 結果の保存
    # np.random.seed()
    # file_name = "".join([str(np.random.choice(10)) for x in range(10)])
    file_name = experiment_name

    results = {}

    results["train_losses"] = training_loss
    results["val_losses"] = validation_loss

    model.load_state_dict(best_weights)
    model.eval()

    cutoff_val = 10e-2

    results["rmse_state_train"] = get_rmse(
        model=model,
        idxs=train_idx,
        x=old_x,
        y=y,
        augmented_data=augmented_data,
        seq_length=100,
        device=device,
        scaler=capacity_output_scaler,
    )

    results["rmse_state_val"] = get_rmse(
        model=model,
        idxs=val_idx,
        x=old_x,
        y=y,
        augmented_data=augmented_data,
        seq_length=100,
        device=device,
        scaler=capacity_output_scaler,
    )

    results["rmse_state_test"] = get_rmse(
        model=model,
        idxs=test_idx,
        x=old_x,
        y=y,
        augmented_data=augmented_data,
        seq_length=100,
        device=device,
        scaler=capacity_output_scaler,
    )

    results["file_name"] = file_name
    results["best_val_loss"] = best_val_loss

    pkl.dump(results, open(os.path.join(save_path, file_name + ".pkl"), "wb"))

    torch.save(model.state_dict(), os.path.join(save_path, file_name + ".pt"))
    print("Saved")

Epoch: 1, Training loss: -0.79824, Validation loss: -0.97124, 
Epoch: 2, Training loss: -1.04365, Validation loss: -1.10705, 
Epoch: 3, Training loss: -1.63365, Validation loss: -2.82952, 
Epoch: 4, Training loss: -2.95263, Validation loss: -3.18665, 
Epoch: 5, Training loss: -3.28037, Validation loss: -3.54627, 
Epoch: 6, Training loss: -3.40294, Validation loss: -3.67330, 
Epoch: 7, Training loss: -3.46708, Validation loss: -3.56581, 
Epoch: 8, Training loss: -3.50308, Validation loss: -3.38105, 
Epoch: 9, Training loss: -3.55959, Validation loss: -3.63229, 
Epoch: 10, Training loss: -3.59713, Validation loss: -3.71536, 
Epoch: 11, Training loss: -3.61238, Validation loss: -3.70894, 
Epoch: 12, Training loss: -3.64877, Validation loss: -3.35464, 
Epoch: 13, Training loss: -3.65959, Validation loss: -3.62018, 
Epoch: 14, Training loss: -3.68335, Validation loss: -3.16207, 
Epoch: 15, Training loss: -3.67777, Validation loss: -3.69555, 
Epoch: 16, Training loss: -3.71316, Validation lo

In [67]:
supp_data.shape

torch.Size([9, 7])

In [54]:
torch.save(model, f"{save_path}/model_test.pt")